# Sentiment Analysis

In the blocks of code below, we conduct sentiment analysis on Reddit data.

In [22]:
import praw

# Authenticate

user_agent = "Reddit_Scrapper 1.0 by /u//Sure-Drama5780"
reddit = praw.Reddit(
    client_id = "Bjk_uxsBZlpdVwKTNZlzxQ",
    client_secret = "LaZgrTga58GqmrqtmKoWmohvCSugdw",
    user_agent = user_agent
)

In [25]:
import pandas as pd
import numpy as np
import re

headlines = set()

# Hot topics

for submission in reddit.subreddit("news").hot(limit = None):
    print(submission.title)
    print(submission.id)
    print(submission.author)
    print(submission.created_utc)
    print(submission.score)
    print(submission.upvote_ratio)
    print(submission.url)
    break
    headlines.add(submission.title)
print(len(headlines))

9th suspect arrested on murder charge in high school student's beating death
181atky
GenXguy-
1700664627.0
9743
0.97
https://www.cnn.com/jonathan-lewis-beating-death-ninth-suspect-arrested/index.html
0


In [26]:
# Number of people talking about the topic

for submission in reddit.subreddit("news").hot(limit = None):
    headlines.add(submission.title)
print(len(headlines))

226


In [31]:
reddit_df = pd.DataFrame(headlines)
reddit_df.head()

,0
0,WHO makes loneliness a global health priority ...
1,100 officers deployed after Trudeau surrounded...
2,Hamas agrees to tentative deal to free dozens ...
3,Frozen first as Boeing 787 Dreamliner makes la...
4,Iowa teen convicted in beating death of Spanis...


In [32]:
reddit_df.columns = ["Titles"]
reddit_df.head()

,Titles
0,WHO makes loneliness a global health priority ...
1,100 officers deployed after Trudeau surrounded...
2,Hamas agrees to tentative deal to free dozens ...
3,Frozen first as Boeing 787 Dreamliner makes la...
4,Iowa teen convicted in beating death of Spanis...


In [33]:
reddit_df.Titles.duplicated().sum()

0

In [35]:
# Clean the reddit posts
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0–9]+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/\S+', '', text)
    text = re.sub(r':', '', text)
    return text

reddit_df['Titles'] = reddit_df["Titles"].apply(cleanTxt)

reddit_df.head()

,Titles
0,WHO makes loneliness a global health priority ...
1,100 officers deployed after Trudeau surrounded...
2,Hamas agrees to tentative deal to free dozens ...
3,Frozen first as Boeing 787 Dreamliner makes la...
4,Iowa teen convicted in beating death of Spanis...


In [37]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               u"\U00002500-\U00002BEF"
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

reddit_df["Titles"] = reddit_df["Titles"].apply(remove_emoji)
reddit_df.head()

,Titles
0,WHO makes loneliness a global health priority ...
1,100 officers deployed after Trudeau surrounded...
2,Hamas agrees to tentative deal to free dozens ...
3,Frozen first as Boeing 787 Dreamliner makes la...
4,Iowa teen convicted in beating death of Spanis...


In [39]:
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS

def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

reddit_df['Subjectivity'] = reddit_df['Titles'].apply(getSubjectivity)
reddit_df['Polarity'] = reddit_df['Titles'].apply(getPolarity)

reddit_df.head()

,Titles,Subjectivity,Polarity
0,WHO makes loneliness a global health priority ...,0.173737,0.056566
1,100 officers deployed after Trudeau surrounded...,0.000000,0.000000
2,Hamas agrees to tentative deal to free dozens ...,0.800000,0.400000
3,Frozen first as Boeing 787 Dreamliner makes la...,0.333333,0.250000
4,Iowa teen convicted in beating death of Spanis...,0.000000,0.000000


In [40]:
def getInsight(score):
    if score < 0:
        return "Negative"
    elif score == 0:
        return "Neutral"
    else:
        return "Positive"

reddit_df['Insight'] = reddit_df["Polarity"].apply(getInsight)
reddit_df.head()

,Titles,Subjectivity,Polarity,Insight
0,WHO makes loneliness a global health priority ...,0.173737,0.056566,Positive
1,100 officers deployed after Trudeau surrounded...,0.000000,0.000000,Neutral
2,Hamas agrees to tentative deal to free dozens ...,0.800000,0.400000,Positive
3,Frozen first as Boeing 787 Dreamliner makes la...,0.333333,0.250000,Positive
4,Iowa teen convicted in beating death of Spanis...,0.000000,0.000000,Neutral
